# Visualizaciones geográficas

El objetivo de esta notebook es ver proveer una forma sencilla de crear visualizaciones geográficas. Dado que el 99% de los datos son sobre Brasil, se filtraran por los datos prevenientes de dicho país.

### Estructura:
- [Obtener coordenadas](#Obtener-coordenadas)
- [Ejemplos de visualizaciones](#Ejemplos-de-visualizaciones)
- [Mapa coroplético](#Mapa-coroplético)

In [1]:
import pandas as pd
import os.path

%matplotlib inline

In [2]:
%run limpieza.ipynb
df = get_clean_df()

## Obtener coordenadas

Para generar un 'heatmap' con las ciudades necesitamos sus coordenadas. Para ello se descargó un set de datos de http://www.geonames.org con las ciudades de Brasil (http://download.geonames.org/export/dump/BR.zip) que contiene el nombre de las ciudades y sus coordenadas.
Este dataset contiene los siguientes atributos:
- geonameid         : integer id of record in geonames database
- name              : name of geographical point (utf8) varchar(200)
- asciiname         : name of geographical point in plain ascii characters, varchar(200)
- alternatenames    : alternatenames, comma separated, ascii names automatically transliterated, convenience attribute from alternatename table, varchar(10000)
- latitude          : latitude in decimal degrees (wgs84)
- longitude         : longitude in decimal degrees (wgs84)
- feature class     : see http://www.geonames.org/export/codes.html, char(1)
- feature code      : see http://www.geonames.org/export/codes.html, varchar(10)
- country code      : ISO-3166 2-letter country code, 2 characters
- cc2               : alternate country codes, comma separated, ISO-3166 2-letter country code, 200 characters
- admin1 code       : fipscode (subject to change to iso code), see exceptions below, see file admin1Codes.txt for display names of this code; varchar(20)
- admin2 code       : code for the second administrative division, a county in the US, see file admin2Codes.txt; varchar(80) 
- admin3 code       : code for third level administrative division, varchar(20)
- admin4 code       : code for fourth level administrative division, varchar(20)
- population        : bigint (8 byte int) 
- elevation         : in meters, integer
- dem               : digital elevation model, srtm3 or gtopo30, average elevation of 3''x3'' (ca 90mx90m) or 30''x30'' (ca 900mx900m) area in meters, integer. srtm processed by cgiar/ciat.
- timezone          : the iana timezone id (see file timeZone.txt) varchar(40)
- modification date : date of last modification in yyyy-MM-dd format

De todas estas nosotros utilizaremos únicamente name, asciiname, population, latitude y longitude.

In [3]:
columnas = ['geonameid', 'name', 'asciiname', 'alternatenames', 'latitude', 'longitude', 'feature class', 'feature code',
            'country code', 'cc2', 'admin1 code', 'admin2 code', 'admin3 code', 'admin4 code', 'population', 'elevation',
            'dem', 'timezone', 'modification date']
df2 = pd.read_csv(os.path.join('datasets', 'BR.txt'), sep='\t', header=None, names=columnas, usecols=['name', 'asciiname', 'feature class','latitude', 'longitude', 'population'])

Este set de datos viene con varias entradas por ciudad con distintas ubicaciones dentro de las mismas:

In [4]:
df2[df2['name'] == 'Rio de Janeiro'].head()

,name,asciiname,latitude,longitude,feature class,population
33210,Rio de Janeiro,Rio de Janeiro,-22.25000,-42.50000,A,15993583
33211,Rio de Janeiro,Rio de Janeiro,-22.90642,-43.18223,P,6023699
42179,Rio de Janeiro,Rio de Janeiro,-18.06667,-45.01667,H,0
42180,Rio de Janeiro,Rio de Janeiro,-11.84252,-45.17394,H,0
75734,Rio de Janeiro,Rio de Janeiro,-22.92008,-43.33069,A,6323037


Esto se debe a que ciertos registros se refieren a ciudades, y otros parte del terreno que llevan el mismo nombre. Un ejemplo claro es el siguiente registro:

In [5]:
df2.iloc[[42179]]

,name,asciiname,latitude,longitude,feature class,population
42179,Rio de Janeiro,Rio de Janeiro,-18.06667,-45.01667,H,0


Si buscamos esas coordendas encontraremos que se refiere al rio 'Rio de Janeiro'. Es por esto que a su vez utilizaremos el atributo 'feature class' y tomaremos solo aquellas con el valor 'P' (que indica registros de ciudades, pueblos, etc.).

In [6]:
df2 = df2[df2['feature class'] == 'P']

Sin embargo notamos que sigue habiendo varias entradas repetidas:

In [7]:
df2.shape[0] == df2['name'].unique().size

False

Para quedarnos con una sola tomaremos como criterio la que tenga una población mayor.

In [8]:
indices = df2.groupby('name', as_index=True)['population'].idxmax()
df2 = df2.loc[indices]

Chequeamos que ahora los nombres sean únicos:

In [9]:
df2.shape[0] == df2['name'].unique().size

True

Y dado que ya no necesitamos los atributos 'population' ni 'feature class', los borramos:

In [10]:
df2 = df2.drop(columns=['population', 'feature class'])

In [11]:
df2.head()

,name,asciiname,latitude,longitude
23587,Aba,Aba,-6.71667,-37.98333
23577,Aba da Serra,Aba da Serra,-5.91667,-39.51667
104680,Abacabal,Abacabal,-6.64503,-69.83667
23583,Abacate,Abacate,-1.11667,-49.65000
23580,Abacaxis,Abacaxis,-3.91667,-58.75000


Ahora vamos a preparar el dataframe del que queremos hacer el heatmap

In [12]:
df = df[df['country'] == 'Brazil']
df = pd.DataFrame(df['city'].value_counts())
df.columns=['peso']
df = df[df['peso'] > 0]
df.head()

,peso
São Paulo,11711
Rio de Janeiro,3538
Belo Horizonte,2568
Salvador,2314
Brasília,1530


Luego vamos a encontrar la latitud y longitud de las ciudades en nuestro dataframe.

In [13]:
df3 = df.reset_index().merge(df2, left_on='index', right_on='name', how='left').set_index('index')
df3 = df3.drop(columns=['name', 'asciiname'])

Aunque encontramos algunos registros que no tienen las coordenadas asignadas:

In [14]:
df3[pd.isnull(df3['latitude'])].head()

,peso,latitude,longitude
index,,,
Carapicuiba,296,NaN,NaN
Sao Goncalo,239,NaN,NaN
Jaboatao dos Guararapes,204,NaN,NaN
Santo Antonio de Jesus,160,NaN,NaN
Sumare,154,NaN,NaN


Esto puede ocurrir por los tildes o caracteres especiales del portugués, por lo cual ahora realizaremos una búsqueda por la columna 'asciiname'.

In [15]:
df4 = df.reset_index().merge(df2, left_on='index', right_on='asciiname', how='left').set_index('index')
df4 = df4.drop(columns=['name', 'asciiname'])

Ahora queda unir ambos dataframes y ver que ciudades no se encontraron.

In [16]:
df5 = df4.combine_first(df3)

In [17]:
df5[pd.isnull(df5['latitude'])]

,peso,latitude,longitude
index,,,
São Miguel do Oeste,12,NaN,NaN


Acá notamos que es solo un registro!

Buscando manualmente encontramos que en el set de datos con las coordenadas el nombre de esta ciudad es ligeramente distinto: Sao Miguel D'Oeste, y por eso no fue encontrado.
Dado que es uno solo podemos setear sus coordenadas manualmente.

In [18]:
df5.loc['São Miguel do Oeste', 'latitude'] = -26.71868
df5.loc['São Miguel do Oeste', 'longitude'] = -53.5194

## Ejemplos de visualizaciones

In [19]:
import folium
from folium.plugins import MarkerCluster, HeatMap, FastMarkerCluster

Mostramos un mapa con las ciudades donde hay **al menos una visita**.

In [20]:
m = folium.Map(tiles='CartoDB positron', location=[-10.656360, -51.767393], zoom_start=4)
marker_cluster = MarkerCluster().add_to(m)
for reg in df5.itertuples():
    folium.Marker([reg[2], reg[3]], popup=folium.Popup('{}, frecuencia: {}'.format(reg[0], reg[1]), parse_html=True)).add_to(marker_cluster)
m

Un **heatmap** que muestra la frecuencia:

In [21]:
import gmaps

In [22]:
with open('api_key.txt') as f:
    gmaps.configure(api_key=f.readline())

In [23]:
gmaps.configure(api_key='')

In [24]:
fig = gmaps.figure(map_type='HYBRID', layout={'height':'500px'})
heatmap_layer = gmaps.heatmap_layer(
    df5[['latitude', 'longitude']], weights=df5['peso'], point_radius=10.0
)
fig.add_layer(heatmap_layer)

In [25]:
heatmap_layer.max_intensity = None
heatmap_layer.point_radius = 12
heatmap_layer.opacity = 1

In [26]:
heatmap_layer.point_radius = 20

In [27]:
fig

Figure(layout=FigureLayout(height='500px'))

## Mapa coroplético

A continuación vamos a crear un mapa coroplético usando folium. Como datos a representar utilizaremos las frecuencias de las regiones en Brasil.

Para ello primero debemos preparar los datos:

In [28]:
df = get_clean_df()
df.head()

,peso
São Paulo,11711
Rio de Janeiro,3538
Belo Horizonte,2568
Salvador,2314
Brasília,1530


In [29]:
df = df[df['country'] == 'Brazil']
df['region'].cat.remove_unused_categories(inplace=True)
df = df['region'].value_counts()

KeyError: 'country'

In [ ]:
df.head()

Para generar el mapa utilizaremos un archivo GeoJson de los estados de Brasil. El mismo se descargó de https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson.

In [ ]:
import branca

def generate_map(color_scale, line_color):
    
    state_geo = os.path.join('datasets', 'br-states.json')
    m = folium.Map(tiles='CartoDB positron', location=[-10.656360, -51.767393], zoom_start=4)
    
    colorscale = getattr(branca.colormap.linear, color_scale).scale(0, df.max())

    def style_function(feature):
        try:
            peso = df[feature['properties']['name']]
        except KeyError:
            peso = 0
        return {
            'fillOpacity': 0.5,
            'weight': 0.2,
            'color': line_color,
            'fillColor': colorscale(peso)
        }

    def highlight_function(feature):
        return {
            'fillOpacity': 0.7,
        }

    tooltip = folium.GeoJsonTooltip(fields=['name'], aliases=['Estado'])

    folium.GeoJson(state_geo, style_function=style_function, highlight_function=highlight_function, tooltip=tooltip).add_to(m)
#     folium.GeoJson(state_geo).add_to(m)

    colorscale.caption = 'Escala'
    m.add_child(colorscale)
    m.add_child(folium.plugins.Fullscreen(position='bottomright'))
    
    return m

In [ ]:
generate_map('YlOrRd_05', 'orange')

In [ ]:
generate_map('YlGnBu_04', '#42dcf4')

Una **combinación** de ambos:

In [ ]:
m = generate_map('YlOrRd_05', 'orange')
marker_cluster = MarkerCluster().add_to(m)
for reg in df5.itertuples():
    folium.Marker([reg[2], reg[3]], popup=folium.Popup('{}, frecuencia: {}'.format(reg[0], reg[1]), parse_html=True)).add_to(marker_cluster)
m